# **¿Por qué una Base de Datos Vectorial?**

Hasta el momento hemos almacenado nuestros chunks extraidos de los PDF's en un archivo .json, pero esto se vuelve ineficiente cuando tenemos una gran cantidad de chunks. Por ejemplo si tuvieramos mas de 100.000 chunks y quisiéramos encontrar el más similar a una pregunta, tendríamos que calcular la similitud de la pregunta con cada uno de los 100,000 chunks. Sería muy lento.

Vamos a usar una base de datos vectorial ChromaDB. Utiliza algoritmos de indexación muy avanzados (como HNSW) que organizan los vectores de tal manera que puede encontrar los "vecinos más cercanos" a un nuevo vector casi instantáneamente, sin tener que comparar con todos los demás. Es lo que necesitamos para hace posible la búsqueda semántica a gran escala.

In [1]:
import chromadb
import os

db_path = "../db" 
collection_name = "jurisprudencia_peruana"

print(f"Inicializando base de datos en: {db_path}")
client = chromadb.PersistentClient(path=db_path)

print(f"Obteniendo o creando la colección: {collection_name}")
collection = client.get_or_create_collection(name=collection_name)

Inicializando base de datos en: ../db
Obteniendo o creando la colección: jurisprudencia_peruana


Una base de datos puede tener múltiples "colecciones". En una base de datos vectorial una colección es el equivalente a una tabla en una base de datos tradicional. La linea `collection = client.get_or_create_collection(name=collection_name)` se conecta a la colección de `jurisprudencia_peruana` si existe, si no existe lo crea.

Cargamos nuestro corpus vectorizado

In [3]:
import json
from tqdm import tqdm
import os 

corpus_path = "../output/corpus_vectorizado.json" 

with open(corpus_path, 'r', encoding='utf-8') as f:
    corpus = json.load(f)

In [4]:
all_embeddings = []
all_documents = []
all_metadatas = []
all_ids = []

for doc in tqdm(corpus, desc="Procesando documentos para DB"):
    doc_metadata = doc.get('metadatos', {})
    doc_id_base = doc.get('archivo', '') 

    for i, chunk in enumerate(doc['chunks']):
        chunk_id = f"{doc_id_base}_chunk_{i}"
        
        chunk_metadata = doc_metadata.copy() 
        chunk_metadata['archivo'] = doc_id_base 
        
        all_embeddings.append(chunk['embedding'])
        all_documents.append(chunk['text'])
        all_metadatas.append(chunk_metadata)
        all_ids.append(chunk_id)

Procesando documentos para DB: 100%|██████████| 10/10 [00:00<?, ?it/s]


In [11]:
len(all_embeddings), len(all_documents), len(all_metadatas), len(all_ids)

(60, 60, 60, 60)

In [14]:
print(f"\nIndexando {len(all_ids)} chunks en la base de datos...")
collection.add(
    embeddings=all_embeddings,
    documents=all_documents,
    metadatas=all_metadatas,
    ids=all_ids
)

count = collection.count()
print("\n¡Indexación completada!")
print(f"La colección '{collection_name}' ahora contiene {count} chunks.")


Indexando 60 chunks en la base de datos...

¡Indexación completada!
La colección 'jurisprudencia_peruana' ahora contiene 60 chunks.


In [15]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

In [16]:
db_path = "../db"
collection_name = "jurisprudencia_peruana"

print(f"Conectándose a la base de datos en: {db_path}")
client = chromadb.PersistentClient(path=db_path)

# Obtener la colección existente
print(f"Obteniendo la colección: {collection_name}")
collection = client.get_collection(name=collection_name)

Conectándose a la base de datos en: ../db
Obteniendo la colección: jurisprudencia_peruana


In [17]:
def search_relevant_chunks(query_text, num_results=3):
    """
    Busca en la colección de ChromaDB los chunks más relevantes para un texto de consulta.
    """
    query_embedding = model.encode(query_text).tolist()

    results = collection.query(
        query_embeddings=[query_embedding], 
        n_results=num_results
    )
    
    return results

In [20]:

if __name__ == '__main__':
    mi_pregunta = "¿cuál es la razón principal por la que la Corte Suprema decide que al demandante solo le corresponden los beneficios del convenio colectivo durante el tiempo que tuvo contratos de locación de servicios y no durante el periodo que tuvo un contrato CAS?"

    print(f"\nBuscando resultados para la consulta: \n'{mi_pregunta}\n\n'")
    
    search_results = search_relevant_chunks(mi_pregunta, num_results=3)
    
    retrieved_docs = search_results['documents'][0]
    retrieved_metadatas = search_results['metadatas'][0]
    retrieved_distances = search_results['distances'][0]

    print("\n--- Resultados Encontrados ---")
    for i in range(len(retrieved_docs)):
        print(f"\nResultado {i+1}:")
        print(f"  Metadatos de Origen: {retrieved_metadatas[i]}")
        print(f"  Distancia: {retrieved_distances[i]:.4f} (menor es mejor)")
        print(f"  Texto del Chunk: '{retrieved_docs[i]}'")
        
        print("-" * 25)


Buscando resultados para la consulta: 
'¿cuál es la razón principal por la que la Corte Suprema decide que al demandante solo le corresponden los beneficios del convenio colectivo durante el tiempo que tuvo contratos de locación de servicios y no durante el periodo que tuvo un contrato CAS?

'

--- Resultados Encontrados ---

Resultado 1:
  Metadatos de Origen: {'materia': 'Relación laboral indeterminado y otros', 'lugar': 'Moquegua', 'n_casacion': '51460-2022', 'archivo': 'Resolucion_S_N_2025-01-09 15_21_00.360,id=1007485496.pdf'}
  Distancia: 3.6904 (menor es mejor)
  Texto del Chunk: 'CONSIDERANDO: 
Primero. Antecedentes del caso: 
a) 
Pretensión: Conforme se aprecia de la demanda, que corre en autos, la parte 
demandante solicita que se declare la desnaturalización de contratos de 
locación de servicios y contratos CAS suscritos con la demandada, 
consecuentemente 
se 
le 
reconozca 
una 
relación 
laboral 
a 
plazo 
indeterminado bajo el régimen privado. Asimismo, solicita su niv